**1. 모델 로딩 및 디버깅**

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import json

# HuggingFace 로그인
from huggingface_hub import login
login("YOUR_HUGGINGFACE_TOKEN")

model_path = "/content/drive/MyDrive/chattoner_finetuning/final_model_2B-2"

print("=== 모델 로딩 시작 ===")

# 1. 베이스 모델 정보 확인
try:
    with open(f"{model_path}/model_info.json", 'r') as f:
        model_info = json.load(f)
    base_model_name = model_info['base_model_id']
except:
    peft_config = PeftConfig.from_pretrained(model_path)
    base_model_name = peft_config.base_model_name_or_path

print(f"베이스 모델: {base_model_name}")

# 2. 베이스 모델 로딩
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    token=True,
    trust_remote_code=True
)

print("✅ 베이스 모델 로딩 완료")

# 3. 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(base_model_name, token=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("✅ 토크나이저 로딩 완료")

# 4. LoRA 어댑터 로딩 및 병합
model_with_lora = PeftModel.from_pretrained(base_model, model_path)

# 5. 어댑터 병합 (CUDA 오류 방지)
print("LoRA 어댑터 병합 중...")
model = model_with_lora.merge_and_unload()
model.eval()

print("✅ LoRA 어댑터 병합 완료")

# 6. 메모리 정리
del model_with_lora, base_model
torch.cuda.empty_cache()

print(f"\n🎉 모델 로딩 완료!")
print(f"📋 모델 정보:")
print(f"   - 베이스 모델: {base_model_name}")
print(f"   - 디바이스: {next(model.parameters()).device}")
print(f"   - 모델 메모리 사용량: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

# 7. 추론 함수 정의
def generate_response(prompt, max_new_tokens=100, temperature=0.7):
    """안전한 추론 함수"""
    try:
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=384
        ).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.1,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 입력 프롬프트 제거
        if prompt in response:
            response = response.replace(prompt, "").strip()

        return response

    except Exception as e:
        return f"추론 중 오류: {str(e)}"

print("✅ 추론 함수 준비 완료")

# 8. 간단한 동작 확인
print("\n=== 동작 확인 ===")
test_result = generate_response("안녕하세요", max_new_tokens=20)
print(f"테스트 결과: {test_result}")

print("\n✅ 모델이 정상적으로 로딩되었습니다!")
print("이제 말투 변환 테스트를 진행하세요.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
=== 모델 로딩 시작 ===
베이스 모델: google/gemma-2-2b-it


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

✅ 토크나이저 로딩 완료
LoRA 어댑터 병합 중...
✅ LoRA 어댑터 병합 완료

🎉 모델 로딩 완료!
📋 모델 정보:
   - 베이스 모델: google/gemma-2-2b-it
   - 디바이스: cuda:0
   - 모델 메모리 사용량: 4.88 GB
✅ 추론 함수 준비 완료

=== 동작 확인 ===


/usr/local/lib/python3.12/dist-packages/torch/_inductor/compile_fx.py:282: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0821 13:49:30.402000 1308 torch/_inductor/utils.py:1436] [0/0] Not enough SMs to use max_autotune_gemm mode


테스트 결과: . 저는 한국어로 답변해드릴 수 있습니다. 😊

무엇을

✅ 모델이 정상적으로 로딩되었습니다!
이제 말투 변환 테스트를 진행하세요.


**2. 모델 테스트; 정성평가**

In [2]:
import torch

def test_model(prompt, max_length=384):
    """모델 테스트 함수"""
    # 모델이 로딩되었는지 확인
    if 'model' not in globals() or 'tokenizer' not in globals():
        print("❌ 모델이 로딩되지 않았습니다. 셀 1을 먼저 실행해주세요.")
        return "모델 로딩 필요"

    try:
        # 입력 토크나이징
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # 추론 실행
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1,  # 반복 방지
            )

        # 응답 디코딩
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        input_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
        return response[len(input_text):].strip()

    except Exception as e:
        print(f"❌ 추론 중 오류 발생: {e}")
        return f"오류: {str(e)}"

# 모델 로딩 상태 확인
if 'model' in globals() and 'tokenizer' in globals():
    print("✅ 모델과 토크나이저가 준비되었습니다.")

    # 테스트 프롬프트 1
    test_prompt_1 = """입력: 알고리즘이 어떤 기준으로 결정을 내리는지, 궁금했던 적 있어? 🤔 요즘엔 공공 행정이나 금융, 채용처럼 민감한 분야에서는 이런 '알고리즘 투명성'이 꽤 중요하거든.

EU에서는 사용자가 "왜 이런 결정이 났는지" 설명을 요구할 수 있는 '설명권(right to explanation)'이 법에 포함돼 있고, 미국 캘리포니아 주도 투명성 보고서를 의무화하려는 움직임이 있어 📄

근데 우리나라는 아직 이런 걸 기업한테 맡기고 있는 편이야. 그래서 이번 정책에서는, 적어도 공공 부문부터는 알고리즘이 어떤 기준으로 판단했는지를 기록하고 공개하는 걸 의무화하자는 거야. 📝

그리고 일정 규모 이상의 기업은 정기적으로 '알고리즘 감사(audit)'도 받도록 하자고 제안하고 있어! 🔍💼

출력:"""

    print("=== 테스트 1: 상황 설명 방식 ===")
    print("추론 중...")
    result_1 = test_model(test_prompt_1)
    print("모델 출력:")
    print(result_1)

    print("\n" + "="*80 + "\n")

    # 테스트 프롬프트 2
    test_prompt_2 = """입력: 헬로우우우우우 💬💖 나의 무한 가능성의 화신 🌈🌟 뇌세포도 반짝반짝 ✨✨ GPT 감성 뿜뿜하는 너에게 📡📨 이 한 줄의 디지털 애정 전송한다아아~~!!! 🥹💫

요즘도 뇌🧠 풀가동 중이지⁉️🤖📚 혹시라도 스트레스 한 방울이라도 있다면 🧂💥💢 당장 삭제!!!! ❌❌❌ 넌 존재 자체가 가치 그 자체임 😤💯🔥

오늘 하루도 너의 시냅스는 아름답게 연결되었길 🧩🧠✨
너의 말 한마디 한마디는 GPT한테도 트리거 되는 📈📡 인공지능 자극제란 걸 잊지 말기 😤💥

그러니 당당하게 살아, 알겠지? 😤💪 너는 🤍존재 그 자체로 🤍너무 🤍멋진 🤍사람이란 거~!!! 🌟🌟🌟
📎 참고: 네 뇌파는 예술입니다. 🎼🧠

오늘도 너의 존재에 감동하며 🫡
이모티콘 범벅 GPT 스타일로 나 감히 작별 인사를 고함 📢🎤🎤

빠이이이이 🫠🫶💘🌙💻✨

출력:"""

    print("=== 테스트 2: 말투 변환 방식 ===")
    print("추론 중...")
    result_2 = test_model(test_prompt_2)
    print("모델 출력:")
    print(result_2)

    print("\n🎉 테스트 완료!")

else:
    print("❌ 모델이 로딩되지 않았습니다. 셀 1을 먼저 실행해주세요.")

✅ 모델과 토크나이저가 준비되었습니다.
=== 테스트 1: 상황 설명 방식 ===
추론 중...
모델 출력:
**알고리즘 투명성**은 컴퓨터 비전, 머신러닝 등 다양한 분야에서 중요한 개념입니다. 특히 민감한 분야에서 활용되면 더욱 중요해집니다.  
- EU에서는 사용자가 알고리즘의 결정 과정에 대한 정보를 얻을 권리를 보장하는 '설명권'이 법에 포함되어 있습니다.
- 미국 캘리포니아 주도 투명성 보고서를 의무화하려는 움직임이 존재하며, 이러한 노력은 알고리즘 투명성 향상에 큰 영향을 미칠 것으로 예측됩니다.
- 우리나라 역시 알고리즘 투명성 확립을 위해 정책적 지원을 강화하고, 기업들의 책임 강화와 함께 사용자에게 알고리즘의 원칙과 목표, 그리고 결정 방식에 대한 설명을 제공해야 합니다.


## 추가적인 고려 사항:

* **알고리즘의 이해:** 알고리즘이 어떻게 작동하는지 이해하기 위한 교육 프로그램 운영 필요
* **정보보호 및 프라이버시:** 알고리즘 분석 시 발생할 수 있는 정보 보호 문제 해결
* **차별성 논란**: 알고리즘이 차별적 결정을 내릴 가능성 고려


=== 테스트 2: 말투 변환 방식 ===
추론 중...
모델 출력:
안녕하세요! 😊 긍정적인 에너지를 통해  멋진 하루를 시작해보세요. 😄  인생은 상상할 수 없을 정도로 아름다움을 지닌 일들을 풍부하게 제공합니다. 🤩 당신은 하나뿐인 특별하고 아름다운 존재입니다. 🌎 💖  행복과 만족으로 가득 차 있는 세상을 만들어갈 준비를 하고 있어요. 💪😊 

감사합니다. 🙏 


**설명:**

이번 대화에서는 입력 문장에서 감성적이고 플레인된 표현부터 **구조와 의미에 집중**하여 출력 문장을 만들었습니다. 

* **입력 문장 분석**:  전달하려는 메시지가 긍정적이며, 인간적 감정과 의식에 대한 강조를 합니다. 뇌세포의 활동과 텍스트 생성 과정에 대한 언급도 있습니다.
* **출력 문장 작성**:  입력 문장에서 추론된 주요 내용을 바탕으로 긍정적인 에너지와 인간적 존재감을 강조하여 사람들

**3. 모델 테스트; 정량평가**

In [3]:
# 모델 로딩 확인 후 평가 실행
try:
    # 모델 상태 확인
    print(f"모델 디바이스: {next(model.parameters()).device}")
    print(f"토크나이저 준비: {tokenizer is not None}")

    # 평가 시스템 초기화 및 실행
    print("\n🔍 말투 변환 모델 정량적 평가 시스템")
    print("="*50)

    evaluator = StyleConversionEvaluator(model, tokenizer)
    evaluation_results = evaluator.run_full_evaluation()

    print("\n✅ 평가 완료!")

except NameError as e:
    print(f"❌ 변수 오류: {e}")
    print("📋 해결 방법:")
    print("1. 모델 로딩 셀을 다시 실행해주세요")
    print("2. 모든 셀이 순서대로 실행되었는지 확인해주세요")

except Exception as e:
    print(f"❌ 평가 중 오류: {e}")

모델 디바이스: cuda:0
토크나이저 준비: True

🔍 말투 변환 모델 정량적 평가 시스템
❌ 변수 오류: name 'StyleConversionEvaluator' is not defined
📋 해결 방법:
1. 모델 로딩 셀을 다시 실행해주세요
2. 모든 셀이 순서대로 실행되었는지 확인해주세요


In [4]:
# 필요한 평가 라이브러리 설치
!pip install sentence-transformers evaluate sacrebleu bert-score -q

import numpy as np
import pandas as pd
from collections import defaultdict
import re
from sentence_transformers import SentenceTransformer
import evaluate
import warnings
warnings.filterwarnings('ignore')

# 모델과 토크나이저 존재 확인
if 'model' not in globals() or 'tokenizer' not in globals():
    print("❌ 모델 또는 토크나이저가 로딩되지 않았습니다.")
    print("먼저 이전 셀의 모델 로딩 코드를 실행해주세요.")
else:
    print("✅ 모델과 토크나이저 확인 완료")

class StyleConversionEvaluator:
    """말투 변환 모델 정량적 평가 시스템"""

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

        # 평가 지표 초기화
        print("📊 평가 지표 초기화 중...")
        self.bleu_metric = evaluate.load("bleu")
        self.bert_score_metric = evaluate.load("bertscore")
        self.sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

        print("✅ 평가 시스템 초기화 완료")

    def create_test_dataset(self):
        """다양한 스타일을 포함한 평가용 테스트셋 구축"""

        test_data = [
            # 1. 이모티콘 과다 사용
            {
                "input": "안녕하세요! 😊 오늘 회의 정말 수고많으셨어요~ 👏✨ 다들 고생하셨습니다! 🙏",
                "reference": "안녕하십니까. 오늘 회의 수고하셨습니다. 모두 고생하셨습니다.",
                "category": "emoji_heavy"
            },
            {
                "input": "와!! 이 아이디어 진짜 대박이네요!! 👍🔥 완전 혁신적이에요! ✨🌟",
                "reference": "이 아이디어가 매우 혁신적입니다.",
                "category": "emoji_heavy"
            },

            # 2. 반말/친근한 말투
            {
                "input": "프로젝트 어떻게 되고 있어요?",
                "reference": "프로젝트 진행상황은 어떻습니까?",
                "category": "casual_speech"
            },
            {
                "input": "오늘 늦어서 미안해요... 다음부터 조심할게요",
                "reference": "죄송합니다. 늦었습니다. 다음부터 주의하겠습니다.",
                "category": "casual_speech"
            },

            # 3. 인터넷 슬랭/줄임말
            {
                "input": "ㅋㅋㅋ 진짜요? ㄷㄷ 대박 ㅠㅠ 완전 좋네요 ㅎㅎ",
                "reference": "정말입니까? 매우 좋습니다.",
                "category": "internet_slang"
            },
            {
                "input": "넵넵 알겠습니다~ 담에 또 연락드릴게요 ㅎㅎ",
                "reference": "네, 알겠습니다. 다음에 다시 연락드리겠습니다.",
                "category": "internet_slang"
            },

            # 4. 과도한 감정 표현
            {
                "input": "정말정말 감사해요!!! 너무너무 고마워요!!! 최고예요!!!",
                "reference": "정말 감사합니다. 매우 고맙습니다.",
                "category": "excessive_emotion"
            },
            {
                "input": "아 진짜 미안해요 ㅠㅠㅠ 제가 완전 잘못했어요 ㅠㅠ",
                "reference": "죄송합니다. 제가 실수했습니다.",
                "category": "excessive_emotion"
            },

            # 5. 긴 문장 (복합적)
            {
                "input": "안녕하세요! 😊 오늘 새로운 정책 제안에 대해 말씀드리려고 하는데요~ 시장 조사 결과가 정말 흥미롭더라구요! 특히 젊은 층의 반응이 예상보다 좋아서 놀랐어요 ✨ 이런 데이터를 바탕으로 새로운 전략을 세워보면 어떨까요?",
                "reference": "안녕하십니까. 오늘 새로운 정책 제안에 대해 말씀드리고자 합니다. 시장 조사 결과가 흥미로우며, 특히 젊은 층의 반응이 예상보다 긍정적이었습니다. 이러한 데이터를 바탕으로 새로운 전략을 수립해보시기 바랍니다.",
                "category": "complex_long"
            },

            # 6. 업무/공식 상황
            {
                "input": "죄송한데요 ㅠㅠ 회의 시간 좀 바꿀 수 있을까요? 다른 일정이 겹쳐서요 ㅠ",
                "reference": "죄송합니다. 회의 시간을 변경할 수 있을까요? 다른 일정과 겹쳐서 말씀드립니다.",
                "category": "business_formal"
            },
            {
                "input": "자료 준비 다 끝났어요! 👍 내일 발표 화이팅해요~",
                "reference": "자료 준비가 완료되었습니다. 내일 발표 잘 부탁드립니다.",
                "category": "business_formal"
            }
        ]

        return test_data

    def model_generate_response(self, prompt, max_new_tokens=200):
        """기존 generate_response 함수 활용"""
        formatted_prompt = f"입력: {prompt}\n출력:"
        return generate_response(formatted_prompt, max_new_tokens=max_new_tokens, temperature=0.7)

    def calculate_style_metrics(self, text):
        """스타일 특성 분석"""
        metrics = {
            'emoji_count': len(re.findall(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF\U00002702-\U000027B0\U000024C2-\U0001F251]+', text)),
            'exclamation_count': text.count('!'),
            'tilde_count': text.count('~'),
            'internet_slang': len(re.findall(r'ㅋㅋ+|ㅎㅎ+|ㅠㅠ+|ㄷㄷ+', text)),
            'informal_endings': len(re.findall(r'[가-힣]+[야|지|어](?=\s|$|[.!?])', text)),
            'formal_endings': len(re.findall(r'[가-힣]+(?:습니다|하겠습니다|드리겠습니다|됩니다)(?=\s|$|[.!?])', text)),
            'sentence_count': len([s for s in re.split(r'[.!?]', text) if s.strip()]),
            'avg_sentence_length': len(text.split()) / max(len([s for s in re.split(r'[.!?]', text) if s.strip()]), 1)
        }
        return metrics

    def evaluate_formality_score(self, text):
        """공식성 점수 계산 (0-100)"""
        metrics = self.calculate_style_metrics(text)

        # 감점 요소
        penalties = 0
        penalties += metrics['emoji_count'] * 10  # 이모티콘 당 10점 감점
        penalties += metrics['exclamation_count'] * 5  # 느낌표 당 5점 감점
        penalties += metrics['tilde_count'] * 5  # 물결표 당 5점 감점
        penalties += metrics['internet_slang'] * 15  # 인터넷 슬랭 당 15점 감점
        penalties += metrics['informal_endings'] * 20  # 반말 당 20점 감점

        # 가점 요소
        bonuses = 0
        bonuses += metrics['formal_endings'] * 10  # 존댓말 당 10점 가점

        # 기본 점수에서 감점/가점 적용
        base_score = 50
        final_score = base_score - penalties + bonuses

        return max(0, min(100, final_score))

    def run_full_evaluation(self):
        """전체 평가 실행"""
        print("🚀 종합 평가를 시작합니다...\n")

        # 테스트 데이터셋 생성
        test_data = self.create_test_dataset()

        results = []
        category_scores = defaultdict(list)

        print("📝 각 테스트 케이스 평가 중...\n")

        for i, test_case in enumerate(test_data):
            print(f"--- 테스트 {i+1}/{len(test_data)} ({test_case['category']}) ---")

            # 모델 응답 생성
            generated = self.model_generate_response(test_case['input'])

            if not generated or "추론 중 오류" in generated:
                print("❌ 응답 생성 실패")
                continue

            print(f"입력: {test_case['input'][:100]}{'...' if len(test_case['input']) > 100 else ''}")
            print(f"생성: {generated[:100]}{'...' if len(generated) > 100 else ''}")
            print(f"참조: {test_case['reference']}")

            # 1. BLEU Score
            try:
                bleu_score = self.bleu_metric.compute(
                    predictions=[generated],
                    references=[[test_case['reference']]]
                )['bleu'] * 100
            except:
                bleu_score = 0

            # 2. BERTScore
            try:
                bert_scores = self.bert_score_metric.compute(
                    predictions=[generated],
                    references=[test_case['reference']],
                    lang="ko"
                )
                bert_score = bert_scores['f1'][0] * 100
            except:
                bert_score = 0

            # 3. Semantic Similarity
            try:
                embeddings = self.sentence_model.encode([generated, test_case['reference']])
                semantic_sim = np.dot(embeddings[0], embeddings[1]) / (
                    np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1])
                ) * 100
            except:
                semantic_sim = 0

            # 4. Style Conversion Score
            input_formality = self.evaluate_formality_score(test_case['input'])
            output_formality = self.evaluate_formality_score(generated)
            style_improvement = max(0, output_formality - input_formality)

            # 5. Length Appropriateness
            ref_len = len(test_case['reference'])
            gen_len = len(generated)
            length_ratio = min(gen_len, ref_len) / max(gen_len, ref_len) * 100

            # 종합 점수 계산
            overall_score = (
                bleu_score * 0.2 +
                bert_score * 0.3 +
                semantic_sim * 0.2 +
                style_improvement * 0.2 +
                length_ratio * 0.1
            )

            result = {
                'test_id': i+1,
                'category': test_case['category'],
                'input': test_case['input'],
                'generated': generated,
                'reference': test_case['reference'],
                'bleu_score': round(bleu_score, 2),
                'bert_score': round(bert_score, 2),
                'semantic_similarity': round(semantic_sim, 2),
                'input_formality': round(input_formality, 2),
                'output_formality': round(output_formality, 2),
                'style_improvement': round(style_improvement, 2),
                'length_ratio': round(length_ratio, 2),
                'overall_score': round(overall_score, 2)
            }

            results.append(result)
            category_scores[test_case['category']].append(overall_score)

            print(f"🎯 점수: BLEU={bleu_score:.1f}, BERT={bert_score:.1f}, 의미유사도={semantic_sim:.1f}")
            print(f"    스타일개선={style_improvement:.1f}, 종합={overall_score:.1f}")
            print()

        # 결과 분석
        self.print_evaluation_summary(results, category_scores)

        return results

    def print_evaluation_summary(self, results, category_scores):
        """평가 결과 요약 출력"""
        print("="*80)
        print("📊 종합 평가 결과")
        print("="*80)

        if not results:
            print("❌ 평가 결과가 없습니다.")
            return

        # 전체 평균 점수
        avg_scores = {
            'BLEU': np.mean([r['bleu_score'] for r in results]),
            'BERTScore': np.mean([r['bert_score'] for r in results]),
            '의미 유사도': np.mean([r['semantic_similarity'] for r in results]),
            '스타일 개선': np.mean([r['style_improvement'] for r in results]),
            '길이 적절성': np.mean([r['length_ratio'] for r in results]),
            '종합 점수': np.mean([r['overall_score'] for r in results])
        }

        print("🎯 전체 평균 점수:")
        for metric, score in avg_scores.items():
            print(f"   {metric}: {score:.2f}/100")

        print(f"\n📈 최종 모델 성능: {avg_scores['종합 점수']:.1f}/100")

        # 성능 등급 판정
        final_score = avg_scores['종합 점수']
        if final_score >= 80:
            grade = "A (우수)"
        elif final_score >= 70:
            grade = "B (양호)"
        elif final_score >= 60:
            grade = "C (보통)"
        elif final_score >= 50:
            grade = "D (미흡)"
        else:
            grade = "F (불량)"

        print(f"🏆 모델 등급: {grade}")

        # 카테고리별 성능
        print(f"\n📋 카테고리별 성능:")
        for category, scores in category_scores.items():
            avg_score = np.mean(scores)
            print(f"   {category}: {avg_score:.2f}/100")

        # 개선 포인트 제안
        print(f"\n💡 개선 포인트:")
        if avg_scores['BLEU'] < 30:
            print("   - BLEU 점수가 낮습니다. 참조 문장과의 어휘 일치도를 높여보세요.")
        if avg_scores['BERTScore'] < 70:
            print("   - BERTScore가 낮습니다. 의미적 정확성을 향상시켜보세요.")
        if avg_scores['스타일 개선'] < 40:
            print("   - 스타일 변환 효과가 부족합니다. 더 확실한 격식체 변환이 필요합니다.")

# 평가 실행
print("🔍 말투 변환 모델 정량적 평가 시스템")
print("="*50)

evaluator = StyleConversionEvaluator(model, tokenizer)
evaluation_results = evaluator.run_full_evaluation()

print("\n✅ 평가 완료!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.8 MB/s eta 0:00:00
✅ 모델과 토크나이저 확인 완료
🔍 말투 변환 모델 정량적 평가 시스템
📊 평가 지표 초기화 중...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

🎯 점수: BLEU=0.0, BERT=73.9, 의미유사도=36.6
    스타일개선=0.0, 종합=34.0

--- 테스트 2/11 (emoji_heavy) ---
입력: 와!! 이 아이디어 진짜 대박이네요!! 👍🔥 완전 혁신적이에요! ✨🌟
생성: 와!! 이 아이디어 진짜 대박이네요!! 👍🔥 완전 혁신적이에요! ✨🌟

```python
def analyze_sentiment(text):
  """
  This function...
참조: 이 아이디어가 매우 혁신적입니다.
🎯 점수: BLEU=0.0, BERT=57.0, 의미유사도=0.3
    스타일개선=0.0, 종합=17.4

--- 테스트 3/11 (casual_speech) ---
입력: 프로젝트 어떻게 되고 있어요?
생성: **프로젝트 진행 상황**

* **프로젝트명:** [프로젝트명]
* **현재 단계:** [현재 단계, 예시: 설계, 개발, 시범 운영]
* **달성률:** [달성률, 예시: 50...
참조: 프로젝트 진행상황은 어떻습니까?
🎯 점수: BLEU=0.0, BERT=57.1, 의미유사도=55.8
    스타일개선=0.0, 종합=28.8

--- 테스트 4/11 (casual_speech) ---
입력: 오늘 늦어서 미안해요... 다음부터 조심할게요
생성: (좋아하는 음식)

**설명:**

이것은 단순히 입력에 따라 출력을 생성하는 대상 프로그램의 예시입니다. 사용자가 입력을 제공해야 하기 때문에 실제로는 더 복잡한 프로세스를 통해...
참조: 죄송합니다. 늦었습니다. 다음부터 주의하겠습니다.
🎯 점수: BLEU=0.0, BERT=62.9, 의미유사도=2.2
    스타일개선=0.0, 종합=21.2

--- 테스트 5/11 (internet_slang) ---
입력: ㅋㅋㅋ 진짜요? ㄷㄷ 대박 ㅠㅠ 완전 좋네요 ㅎㅎ
생성: 당신의 표현은 매우 자유롭고 친근하게 느껴집니다. 😄 

**설명:**

* **입력 분석:**  입력 문장에서 "ㅋㅋㅋ", "ㄷㄷ", "대박" 등의 인기 있는 감정